In [238]:
import pandas as pd
import numpy as np
pd.options.display.max_rows = 50

pd.options.display.float_format = '{:.2f}'.format

In [239]:
df = pd.read_csv("ben_daily.csv")

In [240]:
## Clean the data except for the log volume, date, symbol

clean = df[pd.notnull(df.total_vol_a)].reset_index().drop(['index','CSize','CPrc','OPrc','total_vol','total_vol_m','total_vol_b'],axis = 1)
BAC = clean.loc[clean.symbol == 'BAC'].reset_index().drop(['index'],axis=1)
BAC['log_vol'] = np.log(BAC['total_vol_a'])
BAC = BAC.drop(['total_vol_a'],axis=1)
BAC.head()

,DATE,SYM_ROOT,symbol,log_vol
0,2018/12/31,BAC,BAC,13.50
1,2019/01/02,BAC,BAC,15.55
2,2019/01/03,BAC,BAC,14.87
3,2019/01/04,BAC,BAC,13.99
4,2019/01/07,BAC,BAC,14.98


In [242]:
## Use the rolling average function to get a geometric mean of the last 20 days
BAC['Rolling_Ave'] = BAC.rolling(20).mean()

In [245]:
## Take only the non-null entries (i.e. first 20 entries are useless for model 3)
BAC = BAC[pd.notnull(BAC.Rolling_Ave)].reset_index().drop(['index'],axis=1)

In [248]:
## Manually input the expiry dates, it's not a fully consistent rule and I didn't feel that
## coding this more 'cleverly' would be all too helpful
## Data from https://cdn.cboe.com/resources/options/Cboe2021OPTIONSCalendar.pdf
## https://cdn.cboe.com/resources/options/Cboe2020OPTIONSCalendar.pdf
## https://www.optionseducation.org/getmedia/419cd892-525d-47ae-9b21-8e280fdcf1bf/2019-expiration-calendar.pdf.aspx
## Define y to be the difference between the average and the reported volume as in paper

BAC['y'] = BAC['log_vol'] - BAC['Rolling_Ave']
BAC['options_expiry'] = 0
BAC.options_expiry.loc[ (BAC.DATE == '2019/01/18') | (BAC.DATE == '2019/02/15') | (BAC.DATE == '2019/03/15') | 
(BAC.DATE == '2019/04/18') | (BAC.DATE == '2019/05/17') | (BAC.DATE == '2019/06/21') | 
(BAC.DATE == '2019/07/19') | (BAC.DATE == '2019/08/16') | (BAC.DATE == '2019/09/20') |  
(BAC.DATE == '2019/10/18') | (BAC.DATE == '2019/11/15') | (BAC.DATE == '2019/12/20')] = 1

BAC.options_expiry.loc[(BAC.DATE == '2020/01/17') | (BAC.DATE == '2020/02/21') | (BAC.DATE == '2020/03/20') |
(BAC.DATE == '2020/04/17') | (BAC.DATE == '2020/05/15') | (BAC.DATE == '2020/06/19') | 
(BAC.DATE == '2020/07/17') | (BAC.DATE == '2020/08/21') | (BAC.DATE == '2020/09/18') |  
(BAC.DATE == '2020/10/16') | (BAC.DATE == '2020/11/20') | (BAC.DATE == '2020/12/18')] = 1


BAC.options_expiry.loc[(BAC.DATE == '2021/01/15') | (BAC.DATE == '2021/02/19') | (BAC.DATE == '2021/03/19') |
(BAC.DATE == '2021/04/16') | (BAC.DATE == '2021/05/21') | (BAC.DATE == '2021/06/18') | 
(BAC.DATE == '2021/07/16') | (BAC.DATE == '2021/08/20') | (BAC.DATE == '2021/09/17') |  
(BAC.DATE == '2021/10/15') | (BAC.DATE == '2021/11/19') | (BAC.DATE == '2021/12/17')] = 1

/Users/bengoldman/opt/anaconda3/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


In [249]:
## Sanity check
BAC.loc[BAC.options_expiry == 1]

,DATE,SYM_ROOT,symbol,log_vol,Rolling_Ave,y,options_expiry
13,2019/02/15,BAC,BAC,15.67,14.58,1.08,1
32,2019/03/15,BAC,BAC,15.70,14.44,1.26,1
56,2019/04/18,BAC,BAC,15.06,14.37,0.68,1
76,2019/05/17,BAC,BAC,13.35,13.94,-0.60,1
100,2019/06/21,BAC,BAC,15.96,14.31,1.66,1
119,2019/07/19,BAC,BAC,14.70,14.41,0.29,1
139,2019/08/16,BAC,BAC,15.65,14.44,1.20,1
163,2019/09/20,BAC,BAC,17.17,14.36,2.81,1
183,2019/10/18,BAC,BAC,14.48,14.23,0.24,1
203,2019/11/15,BAC,BAC,14.53,14.10,0.43,1


In [252]:
## Sanity Check
print(BAC.loc[BAC.options_expiry == 1].y.mean())
print(BAC.loc[BAC.options_expiry == 0].y.mean())

0.735437861401373
-0.036142478254965124


In [247]:
## OLS regression of expiry days versus 
## Constant \gamma = 0.7716, R^2 

import statsmodels.formula.api as smf
result = smf.ols(formula = "y ~ options_expiry",data = BAC).fit()
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.058
Model:                            OLS   Adj. R-squared:                  0.056
Method:                 Least Squares   F-statistic:                     44.92
Date:                Tue, 25 Jan 2022   Prob (F-statistic):           4.09e-11
Time:                        00:09:20   Log-Likelihood:                -744.80
No. Observations:                 738   AIC:                             1494.
Df Residuals:                     736   BIC:                             1503.
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                     coef    std err          t      P>|t|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -0.0361      0.025     -1.